### Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import math

### Load data

In [2]:
path = '/Users/jooyong/github_locals/Soccer_Player_Recommendation_Service/data/player_stats(from_whoscored)/Premier_league_player_stats/2010-2011/'
file_list = os.listdir(path)

Premier_matches = pd.DataFrame()
for file in file_list:

    filepath = path + file
    Premier_matches = pd.concat([Premier_matches, pd.read_json(filepath, lines=True)], axis=0)

### High level analysis

In [3]:
Premier_matches.head()

,date,home_name,away_name,home_lineup,away_lineup
0,"Jmos, 09-Apr-11",Wolves,Everton,"[{'Name': 'Wayne Hennessey', 'Position': 'GK',...","[{'Name': 'Tim Howard', 'Position': 'GK', 'Sho..."
1,"Jmos, 09-Apr-11",Blackburn,Birmingham,"[{'Name': 'Paul Robinson', 'Position': 'GK', '...","[{'Name': 'Ben Foster', 'Position': 'GK', 'Sho..."
2,"Jmos, 09-Apr-11",Bolton,West Ham,"[{'Name': 'Jussi Jääskeläinen', 'Position': 'G...","[{'Name': 'Robert Green', 'Position': 'GK', 'S..."
3,"Jmos, 09-Apr-11",Manchester United,Fulham,"[{'Name': 'Tomasz Kuszczak', 'Position': 'GK',...","[{'Name': 'Mark Schwarzer', 'Position': 'GK', ..."
4,"Jmos, 09-Apr-11",Chelsea,Wigan,"[{'Name': 'Petr Cech', 'Position': 'GK', 'Shot...","[{'Name': 'Ali Al Habsi', 'Position': 'GK', 'S..."


In [4]:
Premier_matches.isnull().sum()

date           0
home_name      0
away_name      0
home_lineup    0
away_lineup    0
dtype: int64

In [5]:
Premier_matches.describe()

,date,home_name,away_name,home_lineup,away_lineup
count,392,392,392,392,392
unique,105,20,20,377,362
top,"Jtan, 02-Feb-11",Stoke,Everton,"[{'Name': 'Mark Schwarzer', 'Position': 'GK', ...",[]
freq,12,21,20,2,16


### Player stats analysis

In [6]:
line_up_stats = pd.concat([Premier_matches[['date', 'home_lineup']].rename(columns={'home_lineup':'line_up'}), Premier_matches[['date', 'away_lineup']].rename(columns={'away_lineup':'line_up'})], axis=0).reset_index(drop=True)

In [7]:
line_up_stats.head()

,date,line_up
0,"Jmos, 09-Apr-11","[{'Name': 'Wayne Hennessey', 'Position': 'GK',..."
1,"Jmos, 09-Apr-11","[{'Name': 'Paul Robinson', 'Position': 'GK', '..."
2,"Jmos, 09-Apr-11","[{'Name': 'Jussi Jääskeläinen', 'Position': 'G..."
3,"Jmos, 09-Apr-11","[{'Name': 'Tomasz Kuszczak', 'Position': 'GK',..."
4,"Jmos, 09-Apr-11","[{'Name': 'Petr Cech', 'Position': 'GK', 'Shot..."


Because goal keeper and other field players have different stat features, two dataframe:one for goalkeeper and one for field player should be made.

In [8]:
gk_player_stat_list = []

field_player_stat_list = []

for i in line_up_stats.index:

    date = line_up_stats.iloc[i]['date']

    for player in line_up_stats.iloc[i]['line_up']:

        if player['Position']=='GK':

            player['date']=date

            gk_player_stat_list.append(player)

        else:

            player['date']=date
            
            field_player_stat_list.append(player)



In [9]:
field_player_stat_df = pd.DataFrame(field_player_stat_list)

In [12]:
# position is wrong...position information of player should be saved in other dataset..
Nani_stats_2010_2011 = field_player_stat_df[field_player_stat_df['Name']=='Nani']

In [14]:
Nani_stats_2010_2011.head()

,Name,Position,Shots,ShotsOT,KeyPasses,PA%,AerialsWon,Touches,Rating,Dribbles,...,BlockedShots,Fouls,Passes,Crosses,AccCrosses,LB,AccLB,ThB,AccThB,date
57,Nani,MC,3,1,2,85.7,0,65,7.96,5,...,0,1,42,4,1,2,2,1,1,"Jmos, 09-Apr-11"
189,Nani,DC,7,4,5,95.2,0,64,8.58,3,...,0,0,42,5,2,3,3,0,0,"Jnne, 01-Feb-11"
393,Nani,DC,7,4,5,95.2,0,64,8.58,3,...,0,0,42,5,2,3,3,0,0,"Jnne, 01-Feb-11"
1141,Nani,DC,8,2,2,92.9,0,73,9.29,4,...,0,1,42,11,2,1,1,0,0,"Jmos, 28-Ago-10"
1453,Nani,FW,4,0,2,72.5,0,74,7.25,3,...,0,1,40,11,2,1,0,2,0,"Jtat, 16-Ago-10"
